## Git Clone

In [ ]:
# Clone the repo
!git clone https://<TOKEN>@github.com/java-master007/Adversarial-Representation-Learning-for-Medical-Imaging.git

Cloning into 'Adversarial-Representation-Learning-for-Medical-Imaging'...
remote: Enumerating objects: 542, done.
remote: Counting objects: 100% (542/542), done.
remote: Compressing objects: 100% (387/387), done.
remote: Total 542 (delta 310), reused 370 (delta 142), pack-reused 0
Receiving objects: 100% (542/542), 52.49 MiB | 26.53 MiB/s, done.
Resolving deltas: 100% (310/310), done.


In [ ]:
# Change to correct folder
%cd Adversarial-Representation-Learning-for-Medical-Imaging/

/content/Adversarial-Representation-Learning-for-Medical-Imaging


In [ ]:
! git checkout improvement/segmentation_input

Already on 'improvement/segmentation_input'
Your branch is up to date with 'origin/improvement/segmentation_input'.


In [ ]:
# Install requirements
# Might need to restart on colab
! pip install -r requirements.txt

## Segmentation Training

In [ ]:
%cd MedSegmentation/

/content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSegmentation


In [ ]:
!python train_Unet.py --train_folder /content/Adversarial-Representation-Learning-for-Medical-Imaging/data --val_folder /content/Adversarial-Representation-Learning-for-Medical-Imaging/data --n_epochs 200 --batch_size 1

CUDA is available. Training on GPU
Training Started!

EPOCH 1/200

2022-01-28 16:20:28.791323: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Iteration: 0. Loss: 0.907404899597168. Validation Loss: 0.9026560187339783. Time(mins) 0.055655686060587566

EPOCH 2/200

Iteration: 1. Loss: 0.8992051084836324. Validation Loss: 0.8901054660479227. Time(mins) 0.05749534368515015

EPOCH 3/200

Iteration: 2. Loss: 0.8818599780400594. Validation Loss: 0.8550548752148946. Time(mins) 0.05930732091267903

EPOCH 4/200

Iteration: 3. Loss: 0.8403882185618082. Validation Loss: 0.7989222804705302. Time(mins) 0.06126513878504435

EPOCH 5/200

Iteration: 4. Loss: 0.7782095670700073. Validation Loss: 0.7438917756080627. Time(mins) 0.06313149531682333

EPOCH 6/200

Iteration: 5. Loss: 0.7211084564526876. Validation Loss: 0.6395266652107239. Time(mins) 0.065147304534912

In [ ]:
# Testing different combinations for segmentation
# number of iterations, learning rate, batch size.
import os

iters = [1000, 2000, 4000, 6000, 8000]
lr = [0.01, 0.05]
batch = [1,2,3]

for i in iters:
  for l in lr:
    for b in batch:
      os.system("python train_Unet.py --train_folder /content/drive/MyDrive/Thesis/validationData --val_folder /content/drive/MyDrive/Thesis/validationData --model_checkpoints model_checkpoints_niter_{0}_lr_{1}_batch_{2} --optimizer_checkpoints model_optimizers_niter_{3}_lr_{4}_batch_{5} --experiment_name Experiment_niter_{9}_lr_{10}_batch_{11} --n_epochs {12} --l_rate {13} --batch_size {14}".format(i,l,b,i,l,b,i,l,b,i,l,b))

## Segment Images With Saved Model

In [ ]:
%cd MedSegmentation/

[Errno 2] No such file or directory: 'MedSegmentation/'
/content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSegmentation


In [ ]:
! python api.py --model_dir /content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSegmentation/model_checkpoints --test_images /content/Adversarial-Representation-Learning-for-Medical-Imaging/images --no_eval

CUDA is available. Training on GPU


## Save Model

In [ ]:
# Import files to download zips
from google.colab import files

In [ ]:
# Zip the mlruns metrics to analyse
!zip -r /content/current_model.zip .
files.download("/content/current_model.zip")

In [ ]:
# Zip the best model analysed based on the mlruns
!zip -r /content/best_segmentation_model.zip /content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSegmentation/model_checkpoints
files.download("/content/best_segmentation_model.zip")

## Utils

In [ ]:
! rm -r /content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSegmentation/runs

In [ ]:
! rm -r /content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSegmentation/results

In [ ]:
! rm -r /content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSegmentation/mlruns

In [ ]:
! rm -r /content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSegmentation/model_checkpoints

In [ ]:
! rm -r /content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSegmentation/model_optimizers

## Experiments

In [ ]:
from keras import backend as K
import os
import cv2
from torch.autograd import Variable
import torch

def dice_coeff(y_true, y_pred, smooth=1):
    """
    Calculates the dice coefficient for the images
    @param y_true: The ground truth mask
    @param y_pred: The predicted mask
    @param smooth:
    @return: The Dice Coefficient value
    """

    intersection = K.sum(y_true * y_pred)
    union = K.sum(y_true) + K.sum(y_pred)
    dice = K.mean((2. * intersection + smooth) / (union + smooth))
    return K.get_value(dice)


def jaccard_index(y_true, y_pred, smooth=1):
    """
    Performs jaccard index using jaccard score from scikit learn
    @param smooth:
    @param y_true: The true ground truth value
    @param y_pred: The predicted value
    @return: The Jaccard score between the two images
    """

    intersection = K.sum(K.abs(y_true * y_pred))
    union = K.sum(y_true) + K.sum(y_pred) - intersection
    iou = K.mean((intersection + smooth) / (union + smooth))
    return K.get_value(iou)

def bce_metric(y_true, y_pred):
  loss = torch.nn.BCELoss()
  return loss(y_pred, y_true)

In [ ]:
# Average metric results
dice = 0
jacc = 0
pred_folder = '/content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSegmentation/results'
or_folder = '/content/drive/MyDrive/Thesis/seg_masks'
for i in os.listdir(or_folder):
  t = cv2.imread(os.path.join(or_folder, i), 0)
  p = cv2.imread(os.path.join(pred_folder, i), 0)

  t = t.reshape((t.shape[0], t.shape[1], 1)).transpose((2, 0, 1)).astype(float)/255
  p = p.reshape((p.shape[0], p.shape[1], 1)).transpose((2, 0, 1)).astype(float)/255

  dice += dice_coeff(t, p)
  jacc += jaccard_index(t, p)

print("Dice Coefficient:", dice/len(os.listdir(or_folder)))
print("Jaccard Index:", jacc/len(os.listdir(or_folder)))

Dice Coefficient: 0.03548681480820949
Jaccard Index: 0.018242735536328245


In [ ]:
p = cv2.imread('/content/Adversarial-Representation-Learning-for-Medical-Imaging/MedSegmentation/results/1_mask.png', 0)
t = cv2.imread('/content/drive/MyDrive/Thesis/trainingData/1/1_1_mask.png', 0)

print(p.shape)
print(t.shape)

(2457, 1996)
(2457, 1996)


In [ ]:
p = torch.tensor(p, dtype=torch.float)/255
t = torch.tensor(t, dtype=torch.float)/255

In [ ]:
bce_metric(t,p)

tensor(0.3023)

In [ ]:
dice_coeff(t,p)

0.032773282

In [ ]:
import cv2

In [ ]:
a = cv2.imread('/content/Adversarial-Representation-Learning-for-Medical-Imaging/data/1/1.png')

In [ ]:
a

In [ ]:
! python --version

Python 3.7.12
